# Shor’s Algorithm in Qiskit & Quantum Rings

**Goal:** Factor a composite `N` via quantum order‐finding and continued fractions.

Notebook outline:  
1. Setup & imports  
2. Modular‐addition circuits  
3. Controlled‐modular exponentiation  
4. Assembling the full Shor circuit  
5. Running on AerSimulator  
6. Running on Quantum Rings backend  
7. Results & interpretation

In [2]:
import numpy as np
from math import gcd, ceil, log2
from fractions import Fraction
import random

from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT, CDKMRippleCarryAdder
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler

import QuantumRingsLib
from QuantumRingsLib import AncillaRegister
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2

import time

## 2. Modular‐Addition Primitives

We show two approaches to compute `(x + a) mod N`:
- **QFT‐based**: cheap qubit count but uses non‐native gates  
- **Ripple‐carry adder**: uses the CDKM adder for full reversibility

In [ ]:


shots = 16

def modular_addition(a, N, n):
    qc = QuantumCircuit(n)
    # QFT
    qc.append(QFT(n).to_gate(), range(n))
    # Add 'a' modulo N
    for i in range(n):
        qc.p(2 * np.pi * a * (2 ** i) / N, i)
    # Inverse QFT
    qc.append(QFT(n).inverse().to_gate(), range(n))
    return qc

def quantum_modular_addition_clean(a, N, n_bits):
    """
    Build (x + a) mod N using ripple-carry adder.
    Clean version safe for .to_gate()
    """
    adder = CDKMRippleCarryAdder(n_bits)
    expected_qubits = adder.num_qubits
    qc = QuantumCircuit(expected_qubits)

    # Pre-load constant 'a' into higher bits
    a_bin = format(a, f"0{n_bits}b")
    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(i + n_bits)

    qc.append(adder.to_gate(label=f"+{a} mod {N}"), list(range(expected_qubits)))
    return qc

def add_const_mod_N(a, N, n_bits):
    """
    Adds constant 'a' to a quantum register mod N.
    Returns QuantumCircuit acting on 2n qubits (input + ancilla).
    """
    from qiskit import QuantumCircuit, QuantumRegister

    qc = QuantumCircuit(2 * n_bits)

    # Apply adder for x + a
    adder = CDKMRippleCarryAdder(n_bits)
    a_bin = format(a, f"0{n_bits}b")
    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(i + n_bits)

    qc.append(adder.to_gate(label=f"+{a}").control(0), list(range(2 * n_bits)))

    return qc


def modular_add_const_mod_N(a, N, n_bits):
    """
    Adds a constant 'a' to x (mod N). Result = (x + a) % N.
    Total qubits: 2*n_bits + 1 (for the adder: input, work, and carry)
    plus 1 ancilla flag for comparison.
    """
    from qiskit import QuantumRegister, QuantumCircuit
    from qiskit.circuit.library import CDKMRippleCarryAdder

    # Create registers for input x, work (ancilla), extra carry, and flag
    x = QuantumRegister(n_bits, name="x")
    anc = QuantumRegister(n_bits, name="work")
    carry = QuantumRegister(1, name="carry")
    flag = QuantumRegister(1, name="flag")
    qc = QuantumCircuit(x, anc, carry, flag)

    # Prepare the constant 'a' in the work register
    adder = CDKMRippleCarryAdder(n_bits)
    a_bin = format(a, f"0{n_bits}b")
    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(anc[i])
    # The adder expects inputs on (x, anc, carry): total 2*n_bits+1 qubits
    qc.append(adder.to_gate(label=f"ADD {a}"), x[:] + anc[:] + carry[:])

    # Compare the result in x with N; if x >= N, set flag.
    N_bin = format(N, f"0{n_bits}b")
    for i, bit in enumerate(reversed(N_bin)):
        if bit == '0':
            qc.x(x[i])  # negate bits for comparison
    qc.mcx(x[:], flag[0])
    for i, bit in enumerate(reversed(N_bin)):
        if bit == '0':
            qc.x(x[i])  # undo negation

    # Subtract N if flag==1: prepare subtraction by uncomputing the constant
    subtractor = CDKMRippleCarryAdder(n_bits).inverse()
    for i, bit in enumerate(reversed(N_bin)):
        if bit == '1':
            qc.x(anc[i])
    # The subtractor also expects (x, anc, carry); apply controlled on flag.
    qc.append(subtractor.to_gate(label=f"-{N}").control(1), [flag[0]] + x[:] + anc[:] + carry[:])

    return qc

from qiskit import QuantumCircuit

def controlled_multiply_const_mod_N(a, N, n_bits):
    """
    Constructs a quantum circuit that performs controlled multiplication
    by a constant modulo N.

    Parameters:
        a (int): The constant multiplier.
        N (int): The modulus.
        n_bits (int): Number of bits for the input and result registers.

    Returns:
        QuantumCircuit: A quantum circuit that performs controlled modular multiplication.
    """
    # Create a quantum circuit with input, result, work, and flag qubits
    x = QuantumRegister(n_bits, name="x")      # Input register
    res = QuantumRegister(n_bits, name="res")  # Result register
    work = QuantumRegister(n_bits, name="work")  # Workspace
    flag = QuantumRegister(1, name="flag")     # Ancilla flag qubit
    qc = QuantumCircuit(x, res, work, flag)

    # Perform controlled modular multiplication
    for i in range(n_bits):
        # Compute (a * 2^i) % N
        a_shift = (a * pow(2, i, N)) % N
        if a_shift == 0:
            continue

        # Add (a * 2^i) % N to the result register
        add_circuit = modular_add_const_mod_N(a_shift, N, n_bits)
        add_gate = add_circuit.to_gate(label=f"+{a_shift} mod {N}").control(1)

        # Apply the controlled addition gate, controlled on x[i]
        qc.append(add_gate, [x[i]] + list(res) + list(work) + [flag[0]])

    return qc

def multiply_const_mod_N(a, N, n_bits):
    """
    Quantum circuit for a * x mod N where x is a quantum register.
    Returns a circuit acting on input, work, and ancilla qubits.
    """
    from qiskit import QuantumRegister, QuantumCircuit

    x = QuantumRegister(n_bits, name="x")      # input register
    result = QuantumRegister(n_bits, name="res") # output register
    work = QuantumRegister(n_bits, name="work")  # workspace
    flag = QuantumRegister(1, name="flag")       # ancilla

    qc = QuantumCircuit(x, result, work, flag)

    for i in range(n_bits):
        a_shift = (a * pow(2, i, N)) % N
        if a_shift == 0:
            continue

        add_circuit = modular_add_const_mod_N(a_shift, N, n_bits)
        add_gate = add_circuit.to_gate(label=f"+{a_shift} mod {N}").control(1)

        # Control on x[i], apply to result + work + flag
        qc.append(add_gate, [x[i]] + list(result) + list(work) + [flag[0]])

    return qc

d

## 3. Controlled Modular Exponentiation

We now turn repeated modular additions into a single `ModExp` gate for phase estimation.

In [3]:
def controlled_modular_exponentiation(a, N, n_count):
    """
    Controlled modular exponentiation with .to_gate().control()
    Safe for Quantum Rings execution (no classical ops).
    """
    n_bits = ceil(log2(N))
    adder = CDKMRippleCarryAdder(n_bits)
    work_qubits = adder.num_qubits

    counting_qr = QuantumRegister(n_count, "count")
    working_qr = QuantumRegister(work_qubits, "work")
    qc = QuantumCircuit(counting_qr, working_qr)

    qc.x(working_qr[0])  # Set |1> as initial value

    for i in range(n_count):
        exponent = 2 ** i
        mult = pow(a, exponent, N)

        # Clean modular addition circuit → gate → controlled
        mod_add = quantum_modular_addition_clean(mult, N, n_bits)
        mod_gate = mod_add.to_gate(label=f"add({mult})").control(1)
        qc.append(mod_gate, [counting_qr[i]] + list(working_qr))

    # Return the circuit as a gate, ALREADY implicitly controlled
    return qc.to_gate(label="ModExp")

def build_shors_qc_sampler_ready(N, a, n_count):
    n_bits = ceil(log2(N))
    adder = CDKMRippleCarryAdder(n_bits)
    work_qubits = adder.num_qubits

    counting_qr = QuantumRegister(n_count, "count")
    working_qr = QuantumRegister(work_qubits, "work")
    qr = QuantumRegister(n_count + work_qubits, "q") # Allocate based on actual need
    qc = QuantumCircuit(qr)

    qc.h(qr[:n_count])

    # The controlled_modular_exponentiation gate expects the counting qubits
    # as control and the working qubits as the target.
    modexp_gate = controlled_modular_exponentiation(a, N, n_count)

    print(f"[DEBUG] modexp_gate expects {modexp_gate.num_qubits} qubits, circuit has {len(qr)} qubits.")
    assert modexp_gate.num_qubits == len(qr), "Mismatch: gate and circuit register size don't match."

    # Apply the gate to the correct qubits
    qc.append(modexp_gate, list(qr[:n_count]) + list(qr[n_count:]))

    qc.append(QFT(n_count, inverse=True).to_gate(label="QFT†"), qr[:n_count])

    return qc

def build_shors_qc(N, a, n_count):
    """
    Builds the complete quantum circuit for Shor's algorithm.
    """
    n_bits = ceil(log2(N))
    adder = CDKMRippleCarryAdder(n_bits)
    work_qubits = adder.num_qubits
    print(f"[DEBUG] N={N}, n_bits={n_bits}, work_qubits={work_qubits}")

    counting_qr = QuantumRegister(n_count, "count")
    working_qr = QuantumRegister(work_qubits, "work")
    classical_cr = ClassicalRegister(n_count, "meas")
    qc = QuantumCircuit(counting_qr, working_qr, classical_cr)

    qc.h(counting_qr)

    # Create the modular exponentiation gate
    modexp_gate = controlled_modular_exponentiation(a, N, n_count) # Do NOT apply control here
    print(f"[DEBUG] modexp_gate num_qubits: {modexp_gate.num_qubits}")
    print(f"[DEBUG] len(counting_qr) + len(working_qr): {len(counting_qr) + len(working_qr)}")

    # Append the controlled modular exponentiation gate
    qc.append(modexp_gate, list(counting_qr) + list(working_qr))

    # Apply inverse QFT to the counting register
    qc.append(QFT(n_count, inverse=True).to_gate(), counting_qr[:])

    # Measure the counting register
    qc.measure(counting_qr, classical_cr)

    return qc

def run_shors_on_quantum_rings_qrings(N, a, n_count=8):
    """
    Runs Shor's algorithm circuit on Quantum Rings' backend using Qiskit circuits.

    Parameters:
        N (int): Composite number to factor.
        a (int): Base integer (coprime with N).
        n_count (int): Number of qubits in the phase estimation register.

    Returns:
        dict: Normalized bitstring probabilities from Quantum Rings backend.
    """
    from QuantumRingsLib import QuantumRingsProvider
    from quantumrings.toolkit.qiskit import QrBackendV2

    provider = QuantumRingsProvider(
        token='rings-200.TKfz0NUc5MrEIIqTbvf94Jqm7CnNQssj',
        name='cbjp404@leeds.ac.uk'
    )
    provider.active_account()
    backend = QrBackendV2(provider, num_qubits=32)

    qc = build_shors_qc_sampler_ready(N, a, n_count)
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=shots)
    print("Monitoring job status...")
    while True:
        status = job.status()
        print(f"Job status: {status}")
        if status in [JobStatus.DONE, JobStatus.ERROR, JobStatus.CANCELLED]:
            break
        time.sleep(30)
    
    result = job.result()

    # ✅ Get counts and normalize them into probabilities
    raw_counts = result.get_counts()
    total = sum(raw_counts.values())
    counts = {k: v / total for k, v in raw_counts.items()}

    print("Quantum Rings Results:", counts)
    return counts

def run_shors_with_random_a_qrings(N, n_count=6, n_bits=20, max_trials=100):
    for trial in range(max_trials):
        a = random.randint(2, N - 1)

        if gcd(a, N) != 1:
            factor = gcd(a, N)
            print(f"✔ Found classically: {factor} * {N // factor} = {N} (a = {a})")
            return factor, N // factor, a, None

        print(f"Trial {trial + 1}: Trying a = {a}")
        counts = run_shors_on_quantum_rings_qrings(N, a, n_count)

        measured_bitstring = max(counts, key=counts.get)
        phase = int(measured_bitstring, 2) / (2 ** n_count)
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator

        print(f"Estimated r = {r}")

        if r % 2 == 0 and pow(a, r // 2, N) not in [1, N - 1]:
            x = pow(a, r // 2, N)
            factor1 = gcd(x - 1, N)
            factor2 = gcd(x + 1, N)
            if factor1 not in [1, N] and factor2 not in [1, N]:
                print(f"Success! {factor1} * {factor2} = {N} (a = {a}, r = {r})")
                return factor1, factor2, a, r

        print("Failed for this a.")

    print("No non-trivial factors found after all trials.")
    return None


def shors_algorithm(N, n_count=6, max_trials=100):
    """
    Executes Shor's Algorithm to factor a composite integer N using quantum phase estimation.
    """
    for trial in range(max_trials):
        a = random.randint(2, N - 1)
        if gcd(a, N) != 1:
            factor = gcd(a, N)
            print(f"✔ Found classically: {factor} * {N // factor} = {N} (a = {a})")
            return factor, N // factor, a, None

        print(f"Trial {trial+1}: Trying a = {a}")

        qc = build_shors_qc(N, a, n_count)

        backend = AerSimulator()
        tqc = transpile(qc, backend)
        result = backend.run(tqc, shots=1, memory=True).result()
        measured = result.get_memory()[0]

        phase = int(measured, 2) / 2**n_count
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator

        print(f"Estimated r = {r}")

        if r % 2 == 0 and pow(a, r // 2, N) not in [1, N - 1]:
            x = pow(a, r // 2, N)
            factor1 = gcd(x - 1, N)
            factor2 = gcd(x + 1, N)
            if factor1 not in [1, N] and factor2 not in [1, N]:
                print(f"Success! {factor1} * {factor2} = {N} (a = {a}, r = {r})")
                return factor1, factor2, a, r

        print("Failed for this a.")

    print("No non-trivial factors found after all trials.")
    return None, None, None, None

In [12]:
from fractions import Fraction
from math import gcd
from qiskit import transpile
from qiskit_aer import AerSimulator

def run_shors_single(N, a, n_count, shots):
    """Run one shot of Shor's using AerSimulator and return factors if successful."""
    from qiskit import QuantumCircuit
    qc = build_shors_qc(N, a, n_count)
    backend = AerSimulator()
    tqc = transpile(qc, backend, optimization_level=0)
    result = backend.run(tqc, shots=shots, memory=True).result()

    measured = result.get_memory()[0]
    phase = int(measured, 2) / 2**n_count
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator

    if r % 2 == 0 and pow(a, r // 2, N) not in [1, N - 1]:
        x = pow(a, r // 2, N)
        factor1 = gcd(x - 1, N)
        factor2 = gcd(x + 1, N)
        if factor1 not in [1, N] and factor2 not in [1, N]:
            print(f"Success! {factor1} * {factor2} = {N} (a = {a}, r = {r})")
            return factor1, factor2, a, r
    return None

def run_shors_aer(N, n_count=10, max_trials=20, shots=1):
    for trial in range(max_trials):
        a = random.randint(2, N - 1)
        if gcd(a, N) != 1:
            factor = gcd(a, N)
            print(f"✔ Found classically: {factor} * {N // factor} = {N} (a = {a})")
            return factor, N // factor, a, None

        print(f"Trial {trial + 1}: Trying a = {a}")
        result = run_shors_single(N, a, n_count, shots)
        if result:
            return result
        print("Failed.\n")

    print("❌ No non-trivial factors found after all trials.")
    return None

In [13]:
N = 143 # The number you want to factor
n_count = 8 # Number of qubits for phase estimation (start with a small value)

factor1, factor2, a_used, r_found = run_shors_aer(N=N, n_count=n_count)

if factor1 and factor2:
    print(f"Factorization of N = {N}: {factor1} * {factor2}")
    print(f"Used a = {a_used}, Estimated r = {r_found}")
else:
    print(f"Could not find non-trivial factors of N = {N}.")

Trial 1: Trying a = 36
[DEBUG] N=143, n_bits=8, work_qubits=18
[DEBUG] modexp_gate num_qubits: 26
[DEBUG] len(counting_qr) + len(working_qr): 26
Failed.

Trial 2: Trying a = 140
[DEBUG] N=143, n_bits=8, work_qubits=18
[DEBUG] modexp_gate num_qubits: 26
[DEBUG] len(counting_qr) + len(working_qr): 26
Failed.

✔ Found classically: 13 * 11 = 143 (a = 13)
Factorization of N = 143: 13 * 11
Used a = 13, Estimated r = None


In [ ]:
N = 899
n_count = 10  # Adjust as needed
result = run_shors_with_random_a_qrings(N, n_count)

if result:
    factor1, factor2, a_used, r_found = result
    print(f"\nShor's Algorithm Results for N = {N}:")
    print(f"Factor 1: {factor1}")
    print(f"Factor 2: {factor2}")
    print(f"Used a = {a_used}")
    print(f"Found r = {r_found}")
else:
    print(f"\nShor's Algorithm failed to find factors for N = {N} after maximum trials.")

Trial 1: Trying a = 321
[DEBUG] modexp_gate expects 32 qubits, circuit has 32 qubits.
Monitoring job status...
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE
Job status: JobStatus.DONE


## 7. Results & Interpretation

- We pick the bitstring with highest probability → estimate phase = integer/2ⁿ.  
- Continued fractions → denominator *r*.  
- If *r* even and gcd(a^(r/2)±1, N) nontrivial → we have factors.  

Example output for `N=899` with `n_count=10`: